# **Project 1**

## **Group:**
Leonardo de Lellis Rossi - 261900
Henrique Lima Cará de Oliveira - 091518

[Git Repository](https://github.com/leolellisr/poke_RL)

Images and results are showed with IFrame from IPython.display, but it needs the images stored in the /images/report folder, available in our [git repository](https://github.com/leolellisr/poke_RL). 

Images and Results also are presented with links from LightShot, that doesn't need files.

In [2]:
from IPython.display import IFrame


## **The  problem addressed**
* [Pokémon](https://www.pokemon.com) is a popular Japanese RPG (Role Playing Game) which stands a world championship every year; 
* One single [battle](https://bulbapedia.bulbagarden.net/wiki/Pokémon_battle) of Pokémon has two players. Each player has a 6-Pokémon team; 
* Each Pokémon has:
 * 6 [stats](https://bulbapedia.bulbagarden.net/wiki/Stat) (Health Points, Attack, Defense, Special Attack, Special Defense, Speed). The first 5 are used in the damage calculation. The speed defined which Pokémon moves first in the turn.
  * The Health Points goes from 100% (healthy) to 0% (fainted);
 * 4 possible moves (each with a limited number of uses). Each move has an accuracy, a percentage of success or fail;
 * one [ability](https://bulbapedia.bulbagarden.net/wiki/Ability) that has special effects in the field;
 * one [nature](https://bulbapedia.bulbagarden.net/wiki/Nature) that specifies which stats are higher and which are lower;
 * one [item](https://bulbapedia.bulbagarden.net/wiki/Item), that can  restore Health Points or increase the Power of an Attack.
* The winner of the battle is the player that makes all Pokémon of the oposing team to faint (all oposing Pokémon with health points equals zero, "last man standing" criteria);
* Only one Pokémon of each team can be at the battle field at the same time;
* Every turn, each players select one action: one of the 4 moves of their active Pokémon or [switching](https://bulbapedia.bulbagarden.net/wiki/Recall) for one of other non-fainted Pokémon of their team;

* Pokémon can be summarized as an analyze state (turn) -> take action sequence game. 

* By standard, Pokémon is a stochastic game:
 * One move can have an accuracy value less than 100%, then this move has a probability to be missed;
 * The damage moves (attacks) have the following [damage calculation](https://bulbapedia.bulbagarden.net/wiki/Damage):
  ![Damage](https://wikimedia.org/api/rest_v1/media/math/render/svg/b8c51fed93bb9a80ae8febc13700a40b8a5da402)
  
 where:
  * **[Level](https://bulbapedia.bulbagarden.net/wiki/Level)** (the level of the attacking Pokémon);
  * **A** is the effective Attack stat of the attacking Pokémon if the used move is a physical move, or the effective Special Attack stat of the attacking Pokémon if the used move is a special move;
  * **D** is the effective Defense stat of the target if the used move is a physical move or a special move that uses the target's Defense stat, or the effective Special Defense of the target if the used move is an other special move;
  * **[Power](https://bulbapedia.bulbagarden.net/wiki/Power)** is the effective power of the used move;
  * **Weather** is 1.5 if a Water-type move is being used during rain or a Fire-type move during harsh sunlight, and 0.5 if a Water-type move is used during harsh sunlight or a Fire-type move during rain, and 1 otherwise.
  * **[Critical](https://bulbapedia.bulbagarden.net/wiki/Critical_hit)** has 6.25% chance of occurs and multiplies the damage by 1.5;
  * **random** is a random factor between 0.85 and 1.00 (inclusive):
  * **[STAB](https://bulbapedia.bulbagarden.net/wiki/Same-type_attack_bonus)** is the same-type attack bonus. This is equal to 1.5 if the move's type matches any of the user's types, 2 if the user of the move additionally has the ability Adaptability, and 1 if otherwise;
  * **[Type](https://bulbapedia.bulbagarden.net/wiki/Type)** is the type effectiveness. This can be 0 (ineffective); 0.25, 0.5 (not very effective); 1 (normally effective); 2, or 4 (super effective), depending on both the move's and target's types;
  * **[Burn](https://bulbapedia.bulbagarden.net/wiki/Burn_(status_condition))** is 0.5 (from Generation III onward) if the attacker is burned, its Ability is not Guts, and the used move is a physical move (other than Facade from Generation VI onward), and 1 otherwise.
  * **other** is 1 in most cases, and a different multiplier when specific interactions of moves, Abilities, or items take effect. In this work, this is applied just to Pokémon that has the item **Life Orb**, which multiplies the damage by 1.3.
  
  * **Not** used in this work (equals 1):
   * Targets (for Battles with more than two active Pokémon in the field);
   * Badge ( just applied in Generation II);
   
   

# **MDP formulation and discretization model** 

## Original (stochastic)

We considered our original (stochastic) MDP as a tuple $M = (S, A, \phi, R)$, where:
* **S** is the whole set of possible states. One state **s $\in$ S**  is defined at each turn with 12 battle elements concatenated, that correspond to:
 * [0] Our Active Pokémon index (0: Venusaur,  1: Pikachu, 2: Tauros, 3: Sirfetch'd, 4: Blastoise, 5: Charizard);
 * [1] Opponent Active Pokémon index (0: Eevee,  1: Vaporeon, 2: Leafeon, 3: Sylveon, 4: Jolteon, 5: Umbreon);
 * [2-5] Active Pokémon moves base power (if a move doesn't have base power, default to -1);
 * [6-9] Active Pokémon moves damage multipliers;
 * [10] Our remaining Pokémon;
 * [11] Opponent remaining Pokémon.
 
* **A** is the whole set of possible actions. Our action space is a range [0, 8]. One action **a $\in$ A** is one of the possible choices:
 * [0] 1st Active Pokémon move;
 * [1] 2nd Active Pokémon move;
 * [2] 3rd Active Pokémon move;
 * [3] 4th Active Pokémon move;
 * [4] Switch to 1st next Pokémon;
 * [5] Switch to 2nd next Pokémon;
 * [6] Switch to 3rd next Pokémon;
 * [7] Switch to 4th next Pokémon;
 * [8] Switch to 5th next Pokémon.

When a selected action cannot be executed, we random select another possible action.

* **$\phi$** is a stochastic transition function that occurs from state **s** to state **s'**, by taking an action **a**. The following parameters are part of our  stochastic transition function:
 * Move's accuracy (chance of the move successfully occurs or to fail);
 * Damage calculation: The **[Critical](https://bulbapedia.bulbagarden.net/wiki/Critical_hit)** parameter (6.25% chance of occurs) and the **random** parameter, ranging from 0.85 and 1.00 (inclusive).

* **R** is a set of rewards. A reward **r $\in$ R** is acquired in state **s** by taking the action **a**. The rewards are calculated at the end of the turn. The value of reward **r** is defined by:
 * +Our Active Pokémon current Health Points;
 * -2 if our Active Pokémon fainted;
 * -1 if our Active Pokémon have a [negative status condition](https://bulbapedia.bulbagarden.net/wiki/Status_condition);
 * +Number of remaining Pokémon in our team;
 * -Opponent Active Pokémon current Health Points;
 * +2 if opponent Active Pokémon fainted;
 * +1 if opponent Active Pokémon have a [negative status condition](https://bulbapedia.bulbagarden.net/wiki/Status_condition);
 * -Number of remaining Pokémon in opponent team;
 * +15 if we won the battle;
 * -15 if we lost the battle.
 
### Stochastic Team

Our stochastic team, with each Pokémon, their abilities, natures, items, moves (with base power and accuracy) and possible switches are shown in [Team](https://prnt.sc/1y73c3t).


In [16]:
IFrame("images/report/Team stochastic.png", width=1000, height=600)

## Deterministic

To adapt Pokémon to a deterministic environment, we use Pokémon that cannot receive a critical hit, moves with only 100% accuracy and edit the server code to ignore the random parameter in damage calculation, removing the stochastic transition function $\phi$ from our MDP. Therefore, now our MDP is a tuple $M = (S, A, R)$, where:
* **S** is the whole set of possible states. One state **s $\in$ S**  is defined at each turn with 12 battle elements concatenated, that correspond to:
 * [0] Our Active Pokémon index ;
 * [1] Opponent Active Pokémon index ;
 * [2-5] Active Pokémon moves base power (if a move doesn't have base power, default to -1);
 * [6-9] Active Pokémon moves damage multipliers;
 * [10] Our remaining Pokémon;
 * [11] Opponent remaining Pokémon.
 
* **A** is the whole set of possible actions. Our action space is a range [0, 8] (len: 9). One action **a $\in$ A** is one of the possible choices:
 * [0] 1st Active Pokémon move;
 * [1] 2nd Active Pokémon move;
 * [2] 3rd Active Pokémon move;
 * [3] 4th Active Pokémon move;
 * [4] Switch to 1st next Pokémon;
 * [5] Switch to 2nd next Pokémon;
 * [6] Switch to 3rd next Pokémon;
 * [7] Switch to 4th next Pokémon;
 * [8] Switch to 5th next Pokémon.

When a selected action cannot be executed, we random select another possible action.

* **R** is a set of rewards. A reward **r $\in$ R** is acquired in state **s** by taking the action **a**. The rewards are calculated at the end of each turn. The value of reward **r** is defined by:
 * +Our Active Pokémon current Health Points;
 * -2 if our Active Pokémon fainted;
 * -1 if our Active Pokémon have a [negative status condition](https://bulbapedia.bulbagarden.net/wiki/Status_condition);
 * +Number of remaining Pokémon in our team;
 * -Opponent Active Pokémon current Health Points;
 * +2 if opponent Active Pokémon fainted;
 * +1 if opponent Active Pokémon have a [negative status condition](https://bulbapedia.bulbagarden.net/wiki/Status_condition);
 * -Number of remaining Pokémon in opponent team;
 * +15 if we won the battle;
 * -15 if we lost the battle.
 
### Deterministic Team

Our deterministic team, with each Pokémon, their abilities, natures, items, moves (with base power and accuracy) and possible switches are shown in [Team](https://prnt.sc/1ydn52l).

In [7]:
IFrame("images/report/Team deterministic.png", width=1000, height=600)

## Search space

The features that integrate our states are shown in [this figure](https://prnt.sc/1yfksfe) and below. For a single battle between two players with 6 Pokémon each, we have $1.016.064$ possible states.

Since we have 9 possible actions for each Pokémon, we total $9.144.576$ possibilities for each battle.

In [10]:
IFrame("images/report/possible_states.png", width=920, height=600)

# **The environments built**

The environment used is [Pokémon Showdown](https://play.pokemonshowdown.com), a [open-source](https://github.com/smogon/pokemon-showdown.git) Pokémon battle simulator.

[Example](https://prnt.sc/1ydofwv) of one battle in Pokémon Showdown.


In [12]:
IFrame("images/report/showdownEx.jpg", width=1000, height=1000)

To communicate our agents with Pokémon Showdown we used [poke-env](https://poke-env.readthedocs.io/en/latest/) a Python environment for interacting in pokemon showdown battles.

We used separated Python classes for define the Players that are trained with each method. These classes communicates with Pokémon Showdown and implements the poke-env methods to:
* Create battles;
* Accept battles;
* Send orders (actions) to Pokémon Showdown.

## Original (stochastic)

To speed up the battles, we hosted our own server of Pokemon Showdown in localhost. It requires Node.js v10+.

In [ ]:
git clone https://github.com/smogon/pokemon-showdown.git

After clone the repository, it's needed to create a logs folder in root.

To configure the server, we used the following code:

In [ ]:
cd pokemon-showdown
npm install
cp config/config-example.js config/config.js

To start the server, we use:

In [ ]:
node pokemon-showdown start --no-security

# **Characteristics of  the problem**

* Both of our environments (stochastic and deterministic) are episodic. One state occurs after another;

* Our terminal states are:
 * When all our Pokémon are fainted (we lose);
 * When all opponent Pokémon are fainted (we won).

* As specified before, a reward **r** is calculated at the end of a turn. The value of reward **r** is defined by:
 * +Our Active Pokémon current Health Points;
 * -2 if our Active Pokémon fainted;
 * -1 if our Active Pokémon have a [negative status condition](https://bulbapedia.bulbagarden.net/wiki/Status_condition);
 * +Number of remaining Pokémon in our team;
 * -Opponent Active Pokémon current Health Points;
 * +2 if opponent Active Pokémon fainted;
 * +1 if opponent Active Pokémon have a [negative status condition](https://bulbapedia.bulbagarden.net/wiki/Status_condition);
 * -Number of remaining Pokémon in opponent team;
 * +15 if we won the battle;
 * -15 if we lost the battle.

# **Monte Carlo Control**

We implemented Monte Carlo Control First Visit. 

* We defined 2 defaultDicts, $Q(s,a)$ and $N(s,a)$, to store the value function and the number of occurences of action **a** in state **s**, respectively. Both of them are initialized to zero;
 * Each dict has the keys as the states **s**;
 * The value of a key is an array of the len of action space (len: 9);
 * Each value of an array corresponds to an action in our action space;

* We used a step-size of $\alpha = 1/N[s,a]$;

* In the first move of a turn, we choose a random action;

* We used an $\epsilon$-greedy exploration strategy with $\epsilon = N0/(N0+N[s,:])$. We tested with 3 values for $N0: $ {$10^{-4}$, $10^{-3}$ and $10^{-2}$};

* We use the following $\epsilon$-greedy policy:

In [ ]:
def epsilon_greedy_policy(Q, n0, N):
    def policy_fn(state):
        dimension = Q[state].shape
        greedy_action = np.argmax(Q[state])
        epsilon = n0/(n0+np.sum(N[state]))
        probabilities = np.full(dimension, epsilon / dimension[0])
        probabilities[greedy_action] += 1 - epsilon
        return probabilities
    return policy_fn

* In each turn, we collect our observations to define our state with the embed battle function, getting the 12 battle elements concatenated. The state is converted to a string, that is used as a key to our defaultDicts $Q(s,a)$ and $S(s,a)$;
 
* With the state, we select an action using our policy and send it to Showdown;

* We store the state and selected action to append to an episode array in the next turn with the computed reward;

* At the end of the battle, $Q(s,a)$ and $N(s,a)$ are updated for the first time of each state seen in that battle:
 * $N(s,a)$ is increased by 1 for each action performed in a particular state;
 * The value of $Q(s,a)$ is summed to $\alpha*(G(t)-Q(s,a))$;
 * The return $G(t)$ is given by $ \sum \limits _{i=0} ^{T-1} \gamma^iR(t)$. We used $\gamma = 0,75$;
 * Our policy are then updated with the new values of $Q(s,a)$ and $N(s,a)$;

* We trained our agent against a MaxDamagePlayer, that always selects the move with the greater base power. We trained for $10.000$ battles. At the end, we saved $Q$ and $N$ in json files to future use.

* We validate our solution against a MaxDamagePlayer and against a RandomPlayer, that selects random actions at each turn. We validated for $3.333$ battles against each Player.  

## MC_Control - Stochastic - Results 

The instant values of the graphs of "Rewards in Training", "Wins in Training" and "Memory/CPU usage in Traning" can be seen at https://app.neptune.ai/leolellisr/rl-pokeenv/e/RLPOK-84/dashboard/Compara-es-464d2bfc-e396-4bbd-b8bc-3cf9fe90b55e. 

### MC_Control - Stochastic - Rewards (per step) - Training 

[MC_Control - Stochastic - Rewards (per step) - Training](https://prnt.sc/1ycoapc)

In [11]:
IFrame("images/report/reward_computed_MC_Control.png", width=800, height=600)

### MC_Control - Stochastic - Win percentages (per step) - Training
[ MC_Control - Stochastic - Win percentages (per step) - Training](https://prnt.sc/1ycojai)

In [12]:
IFrame("images/report/win_acc_MC_Control.png", width=1000, height=600)

### MC_Control - Stochastic - Win percentages - Validation

### Versus MaxPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $1262/3333$ battles [this is $37.86\%$ and took $743.18$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $1063/3333$ battles [this is $31.89\%$ and took $743.37$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $1166/3333$ battles [this is $34.98\%$ and took $622.97$ seconds]

### Versus RandomPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $3026/3333$ battles [this is $90.79\%$ and took $841.88$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $3113/3333$ battles [this is $93.4\%$ and took $825.55$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $3130/3333$ battles [this is $93.91\%$ and took $762.68$ seconds]


### MC_Control - Stochastic - Memory and CPU usage (absolute time) - Training

[MC_Control - Stochastic - Memory and CPU usage (absolute time) - Training](https://prnt.sc/1ycopro)

In [4]:
IFrame("images/report/cpu and memory MC_Control.png", width=1000, height=600)


### MC_Control - Stochastic - Value Function $V^*(s)$ 

[MC_Control - Stochastic - Value Function $V^*(s)$]()

## MC_Control - Deterministic - Results 

The instant values of the graphs of "Rewards in Training", "Wins in Training" and "Memory/CPU usage in Traning" can be seen at  https://app.neptune.ai/leolellisr/rl-pokeenv/e/RLPOK-102/dashboard/Compara-es-464d2bfc-e396-4bbd-b8bc-3cf9fe90b55e.

### MC_Control - Deterministic - Rewards (per step) - Training 

[MC_Control - Deterministic - Rewards (per step) - Training](https://prnt.sc/1yd5r9z)

In [3]:
IFrame("images/report/reward_computed_MC_Control_deterministic.png", width=800, height=600)

### MC_Control - Deterministic - Win percentages (per step) - Training
[MC_Control - Deterministic - Win percentages (per step) - Training](https://prnt.sc/1yd603w)

In [4]:
IFrame("images/report/win_acc_MC_Control_deterministic.png", width=1000, height=600)

### MC_Control - Deterministic - Win percentages - Validation

### Versus MaxPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $1380/3333$ battles [this is $41.4\%$ and took $485$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $1359/3333$ battles [this is $40.77\%$ and took $400.49$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $1201/3333$ battles [this is $36.03\%$ and took $400.07$ seconds]

### Versus RandomPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $3052/3333$ battles [this is $91.57\%$ and took $680.85$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $3007/3333$ battles [this is $90.22\%$ and took $825.87$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $2997/3333$ battles [this is $89.92\%$ and took $720$ seconds]

### MC_Control - Deterministic - Memory and CPU usage  (absolute time) - Training

[MC_Control - Deterministic - Memory and CPU usage (absolute time) - Training](https://prnt.sc/1yd64i7)

In [5]:
IFrame("images/report/cpu and memory MC_Control_deterministic.png", width=1000, height=600)

### MC_Control - Deterministic - Value Function $V^*(s)$ 

[MC_Control - Deterministic - Value Function $V^*(s)$]()

# **Monte Carlo Control - Function Approximation**

We implemented the Function Approximation with Monte Carlo Control First Visit. 

* We defined an array $w$ to store the weights of our approximator and a defaultDict $N(s,a)$ to store the number of occurences of action **a** in state **s**. $w$ is initialized randomly and $N(s,a)$ is initialized to zero;
 * $N(s,a)$ has the keys as the states **s**. The value of a key is an array of the len of action space (len: 9) and Each value of an array corresponds to an action in our action space;

* We used a step-size of $\alpha = 1/N[s,a]$;

* In the first move of a turn, we choose a random action;

* We used an $\epsilon$-greedy exploration strategy with $\epsilon = N0/(N0+N[s,:])$. We tested with 3 values for $N0: $ {$10^{-4}$, $10^{-3}$ and $10^{-2}$};

* We use the following functions to define our $\epsilon$-greedy policy and to update the weights $w$:

In [ ]:
# Feature vector
def x(state, action):
    state = np.array(state).astype(float)
    return np.append(state, action)

def q_approx_fn(state, action, w):
    state = np.array(state).astype(float)
    return np.dot(x(state, action), w)
        
def epsilon_greedy_policy(self, w, n0, N):
    # epsilon-greedy policy
    def policy_fn(state):
        epsilon = n0 / (n0 + np.sum(N[str(state)]))
        # let's get the greedy action. Ties must be broken arbitrarily
        q_approx = np.array([q_approx_fn(state, action, w) for action in range(9)])
        greedy_action = np.argmax(q_approx)
        action_pick_probability = np.full(9, epsilon / 9)
        action_pick_probability[greedy_action] += 1 - epsilon
        return action_pick_probability

    return policy_fn

* In each turn, we collect our observations to define our state with the embed battle function, getting the string of 12 battle elements concatenated;
 
* With the state, we select an action using our policy and send it to Showdown;

* We store the state and selected action to append to an episode array in the next turn with the computed reward;

* At the end of the battle, $w$ and $N$ are updated for the first time of each state seen in that battle:
 * $N(s,a)$ is increased by 1 for each action performed in a particular state;
 * The value of $w$ is summed to $\alpha*\delta*x(s,a)$;
 * $\delta$ is calculated by $G(t) -$ q_approx_fn$(s, a, w)$;
 * The return $G(t)$ is given by $ \sum \limits _{i=0} ^{T-1} \gamma^iR(t)$. We used $\gamma = 0,75$;
 * Our policy are then updated with the new values of $w$ and $N(s,a)$;

* We trained our agent against a MaxDamagePlayer, that always selects the move with the greater base power. We trained for $10.000$ battles. At the end, we saved $w$ and $N(s,a)$ in json files to future use.

* We validate our solution against a MaxDamagePlayer and against a RandomPlayer, that selects random actions at each turn. We validated for $3.333$ battles against each Player.  


## MC_Control Function Approximation - Stochastic - Results 

The instant values of the graphs of "Rewards in Training", "Wins in Training" and "Memory/CPU usage in Traning" can be seen at https://app.neptune.ai/leolellisr/rl-pokeenv/e/RLPOK-93/dashboard/Compara-es-464d2bfc-e396-4bbd-b8bc-3cf9fe90b55e. 

### MC_Control  Function Approximation - Stochastic - Rewards (per step) - Training 

[MC_Control  Function Approximation - Stochastic - Rewards (per step) - Training ](https://prnt.sc/1ycovut)

In [13]:
IFrame("images/report/reward_computed_MC_Control_FA.png", width=1000, height=600)

### MC_Control Function Approximation - Stochastic - Win percentages (per step) - Training

[MC_Control Function Approximation - Stochastic - Win percentages (per step) - Training](https://prnt.sc/1ycpfrm)

In [14]:
IFrame("images/report/win_acc_MC_Control_FA.png", width=1000, height=600)


### MC_Control Function Approximation - Stochastic - Win percentages - Validation

### Versus MaxPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $2771/3333$ battles [this is $83.14\%$ and took $521.1$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $2796/3333$ battles [this is $83.89\%$ and took $579.7$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $2778/3333$ battles [this is $83.35\%$ and took $807.72$ seconds]

### Versus RandomPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $3312/3333$ battles [this is $99.37\%$ and took $854.3$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $3303/3333$ battles [this is $99.1\%$ and took $765.18$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $3306/3333$ battles [this is $99.2\%$ and took $654.11$ seconds]

### MC_Control Function Approximation - Stochastic - Memory and CPU usage  (absolute time) - Training

[MC_Control Function Approximation - Stochastic - Memory and CPU usage (absolute time) - Training](https://prnt.sc/1ycpquh)

In [17]:
IFrame("images/report/cpu and memory MC_Control_FA.png", width=1000, height=600)

### MC_Control Function Approximation - Stochastic - Value Function $V^*(s)$ 

[MC_Control Function Approximation - Stochastic - Value Function $V^*(s)$]()

## MC_Control Function Approximation - Deterministic - Results 

The instant values of the graphs of "Rewards in Training", "Wins in Training" and "Memory/CPU usage in Traning" can be seen at  https://app.neptune.ai/leolellisr/rl-pokeenv/e/RLPOK-116/dashboard/Compara-es-464d2bfc-e396-4bbd-b8bc-3cf9fe90b55e.

### MC_Control Function Approximation - Deterministic - Rewards (per step) - Training 

[MC_Control Function Approximation - Deterministic - Rewards (per step) - Training ](https://prnt.sc/1yfmhdt)

In [14]:
IFrame("images/report/reward_computed_MC_Control_FA_deterministic.png", width=1000, height=600)

### MC_Control Function Approximation - Deterministic - Win percentages (per step) - Training

[MC_Control Function Approximation - Deterministic - Win percentages (per step) - Training](https://prnt.sc/1yfmlck)

In [15]:
IFrame("images/report/win_acc_MC_Control_FA_deterministic.png", width=1000, height=600)

### MC_Control Function Approximation - Deterministic - Win percentages - Validation

### Versus MaxPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $2022/3333$ battles [this is $60.67\%$ and took $456.87$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $1982/3333$ battles [this is $59.47\%$ and took $508.82$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $2000/3333$ battles [this is $60\%$ and took $561.88$ seconds]

### Versus RandomPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $3312/3333$ battles [this is $99.37\%$ and took $532.47$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $3306/3333$ battles [this is $99.19\%$ and took $723.62$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $3310/3333$ battles [this is $99.31\%$ and took $539.96$ seconds]

### MC_Control Function Approximation - Deterministic - Memory and CPU usage  (absolute time) - Training

[MC_Control Function Approximation - Deterministic - Memory and CPU usage (absolute time) - Training](https://prnt.sc/1yfmqa3)

In [16]:
IFrame("images/report/cpu and memory MC_Control_FA_deterministic.png", width=1000, height=600)

### MC_Control Function Approximation - Deterministic - Value Function $V^*(s)$ 

[MC_Control Function Approximation - Deterministic - Value Function $V^*(s)$]()

# **QLearning**

* We defined 2 defaultDicts, $Q(s,a)$ and $N(s,a)$, to store the value function and the number of occurences of action **a** in state **s**, respectively. Both of them initialized to zero;
 * each dict has key **s** (state) and respective value **a** (action), which is an array of length 9 (size of our action space);

* We used a step-size of $\alpha = 1/N[s, a]$.

* We used discount factor $\gamma = 0.75$.

* We used an $\epsilon$-greedy exploration strategy with $\epsilon = N0/(N0+N[s, :])$. We've tested using 3 different values for $N0: $ {$10^{-4}$, $10^{-3}$ and $10^{-2}$}.

* We use the following $\epsilon$-greedy policy:

In [ ]:
def pi(self, state):
    epsilon = self.n0 / (self.n0 + np.sum(self.N[state]))
    greedy_action = np.random.choice(np.where(self.Q[state] == self.Q[state].max())[0])
    action_pick_probability = np.full(N_OUR_ACTIONS, epsilon / N_OUR_ACTIONS)
    action_pick_probability[greedy_action] += 1 - epsilon
    return np.random.choice(ALL_OUR_ACTIONS, p=action_pick_probability)

* At every of our turns (steps) in a battle (episode), we call the embed battle function to collect our observations, then we define our state by concatenating 12 battle elements. The concatenated elements are converted to a string that can be used as a key to index our dictionaries $Q(s,a)$ and $N(s,a)$.

* In the very first turn of a battle, we first obtain a random state by calling the embed battle function. Then, we choose our next action by following policy $\pi$: $a = \pi(s)$.

* In each of the subsequent turns until the end of the battle, we will do the following:
 - get the reward **R** and the next state **s'** from the previously executed action;
 - increase $N(s, a)$ by 1 for each action **a** executed while in state **s**;
 - $\alpha$ is calculated as: $\alpha=1/N(s, a)$;
 - then, $Q(s, a)$ is calculated as: $Q(s, a) = Q(s, a) + \alpha * (R + \gamma * \max_{s'}Q(s') - Q(s, a))$;
 - at the end, we do: **s** <- **s'**

In [ ]:
# observe R, S'
reward = self.compute_reward(battle)
next_state = self.embed_battle(battle)
# Q-learning
self.N[self.state][self.action] += 1
alpha = 1.0 / self.N[self.state][self.action]
self.Q[self.state][self.action] += \
    alpha * (reward + self.gamma * np.max(self.Q[next_state]) - self.Q[self.state][self.action])
# S <- S'
self.state = next_state

* We trained our agent against a MaxDamagePlayer, that always selects the move with the greater base power. We trained for $10.000$ battles. At the end, we saved $Q$ and $N$ in json files to future use.

* We validated our solution against a MaxDamagePlayer and against a RandomPlayer, that selects random actions at each turn. We validated for $3.333$ battles against each Player.  

## Q-Learning - Stochastic - Results

The instant values of the graphs of "Rewards in Training", "Wins in Training" and "Memory/CPU usage in Traning" can be seen at https://app.neptune.ai/leolellisr/rl-pokeenv/e/RLPOK-95/dashboard/Compara-es-464d2bfc-e396-4bbd-b8bc-3cf9fe90b55e. 

### Q-Learning - Stochastic - Rewards (per step) - Training

[Q-Learning - Stochastic - Rewards (per step) - Training](https://prnt.sc/1ycpxci)

In [6]:

IFrame("images/report/reward_computed_QLearning.png", width=800, height=600)

### Q-Learning - Stochastic - Win percentages (per step) - Training

[Q-Learning - Stochastic - Win percentages (per step) - Training](https://prnt.sc/1ycq2uv)

In [7]:
IFrame("images/report/win_acc_QLearning.png", width=800, height=600)

### Q-Learning - Stochastic - Win percentages - Validation

### Versus MaxPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $1510/3333$ battles [this is $45.3\%$ and took $845.64$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $1156/3333$ battles [this is $34.68\%$ and took $1038.49$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $1503/3333$ battles [this is $45.09\%$ and took $748.07$ seconds]

### Versus RandomPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $3123/3333$ battles [this is $93.7\%$ and took $957.94$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $2859/3333$ battles [this is $85.78\%$ and took $912.79$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $3021/3333$ battles [this is $90.64\%$ and took $850.87$ seconds]

### Q-Learning - Stochastic - Memory and CPU usage  (absolute time) - Training

[Q-Learning - Stochastic - Memory and CPU usage (absolute time) - Training](https://prnt.sc/1ycqa4f)

In [5]:
IFrame("images/report/cpu and memory QLearning.png", width=800, height=600)

### Q-Learning - Stochastic - Value Function $V^*(s)$ 

[Q-Learning - Stochastic - Value Function $V^*(s)$]()

## Q-Learning - Deterministic - Results 

The instant values of the graphs of "Rewards in Training", "Wins in Training" and "Memory/CPU usage in Traning" can be seen at  https://app.neptune.ai/leolellisr/rl-pokeenv/e/RLPOK-119/dashboard/Compara-es-464d2bfc-e396-4bbd-b8bc-3cf9fe90b55e.


### Q-Learning - Deterministic - Rewards (per step) - Training 

In [18]:
IFrame("images/report/reward_computed_QLearning_deterministic.png", width=800, height=600)

### Q-Learning  - Deterministic - Win percentages (per step) - Training


In [19]:
IFrame("images/report/win_acc_QLearning_deterministic.png", width=800, height=600)

### Q-Learning  - Deterministic - Win percentages - Validation

### Versus MaxPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $1446/3333$ battles [this is $43.38\%$ and took $572.78$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $1077/3333$ battles [this is $32.31\%$ and took $444.55$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $1461/3333$ battles [this is $43.83\%$ and took $582.1$ seconds]

### Versus RandomPlayer
* Player with $N0=0.0001$ and $\gamma=0.75$ won $2801/3333$ battles [this is $84.04\%$ and took $698.99$ seconds]

* Player with $N0=0.001$ and $\gamma=0.75$ won $2800/3333$ battles [this is $84.01\%$ and took $794.75$ seconds]

* Player with $N0=0.01$ and $\gamma=0.75$ won $2934/3333$ battles [this is $88.03\%$ and took $601.89$ seconds]

### Q-Learning - Deterministic - Memory and CPU usage  (absolute time) - Training

In [21]:
IFrame("images/report/QLearningMemory _ CPU usage_deterministic.png", width=800, height=600)

### Q-Learning - Deterministic - Value Function $V^*(s)$ 

[Q-Learning - Deterministic - Value Function $V^*(s)$]()

# Related work

Given the stochasticity of the Pokémon game, the environment proves to be a good alternative to validate more robust reinforcement learning systems. However, in recent years, few reinforcement learning jobs have been willing to use Pokémon battles to validate their models.

Regarding tabular methods, the work of [Rill-García, R. 2018](https://ccc.inaoep.mx/~esucar/Clases-mgp/Proyectos/2018/reinforcement-learning-turn%20%281%29.pdf) can be mentioned. The author proposes the use of the Q-Learning algorithm. The author defines as states only the index of his active Pokémon and the opponent's. However, 301 Pokémon are considered to choose from, among the most used in competitive battles. Actions were also defined from 601 moves most used in competitive battles, plus one, referring to an action that cannot be performed. Rewards were set based on the Pokémon that passed out (their or the opponent's), the percentages of damage produced or taken, and based on an "unable to act" rating. The author used constant values of $\alpha = 0.1$ and $\gamma = 0.8$. The author used 10 different teams for validation and reported an average of $58\%$ wins against a RandomPlayer (player who selects random actions at each turn).

Another tabular method that can be cited is the work of [Kalose, A et. al. 2018](https://web.stanford.edu/class/aa228/reports/2018/final151.pdf). The authors also propose the use of the Q-Learning algorithm, however, it uses two different policies, an $\epsilon-greedy$ and a Softmax. The authors define as states the Health Points of their Pokémon and the opponent's and the types of their Pokémon and the opponent's. The defined actions are the 4 possible moves of each Pokemon. The authors did not establish how the rewards were defined, or the alpha and gamma values used. The authors reported an average of $60\%$ winnings against a RandomPlayer (player who selects random actions each turn) using $\epsilon-greedy$ policy and $68\%$ winnings against a RandomPlayer using Softmax policy.